In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
import matplotlib.pyplot as plt
import base64

In [3]:
## 카카오 맵 
driver = webdriver.Chrome()
driver.get('https://map.kakao.com/')
time.sleep(2)

## input 창에 검색어 입력
searchbox = driver.find_element(By.ID, 'search.keyword.query')
searchbox.send_keys('홍대 맛집'+ Keys.ENTER)
time.sleep(3)

In [4]:
## 상세보기 주소 가져오기 
list_url = []

## 아래 1,2,3,4,5 리스트 번호 다 뜨도록
# 장소 더보기 클릭
driver.find_element(By.ID, 'info.search.place.more').click()
time.sleep(2)
# 1번리스트로 돌아오기
driver.find_element(By.ID, 'info.search.page.no1').click()
time.sleep(2)

In [5]:
for count in range(1):
    
    for inx in range(2, 6):
        
        # 현재 리스트의 상세보기 주소 스크래핑
        moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
        for i in range(len(moreview_element)):
            list_url.append(moreview_element[i].get_attribute('href'))
        # 다음 리스트 클릭
        driver.find_element(By.ID, f'info.search.page.no{inx}').click()
        # 5번 리스트 저장
        if inx == 5:
            moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
            for i in range(len(moreview_element)):
                list_url.append(moreview_element[i].get_attribute('href'))
        time.sleep(2)
    # 다음 리스트 페이지 넘어가기
    driver.find_element(By.ID ,'info.search.page.next').click()
    time.sleep(2)

In [9]:
len(list_url)

15

In [8]:
# 현재 리스트의 상세보기 주소 가져오기
list_url = []
moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
for i in range(len(moreview_element)):
    list_url.append(moreview_element[i].get_attribute('href'))

In [10]:
## 웹페이지 종료
driver.close()

In [26]:
## 정보 넣을 빈 딕셔너리
data = {
    'merchant' : [],
    'category' : [],
    'menu' : [],
    'star' : [],
    'address' : [],
    'oper_time' : [],
    'starCount' : [],
    'reviewCount' : []
}

data2 = {
    'user_name' : [],
    'user_rank' : [],
    'num_response' : [],
    'user_star' : [], 
    'rating' : [],
    'time' : [],
    'content' : []
}

In [25]:
## 상세보기 팝업
driver = webdriver.Chrome()
driver.get(list_url[11])

In [24]:

list_url[11]

'https://place.map.kakao.com/10368349'

In [27]:
# 영업시간 더 보기 클릭
try:
    oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent = "main_info,more_timeinfo"]')
    oper_more[0].click()
    time.sleep(2)  

# 영업시간 더보기가 없는 경우
except:
    pass

# 후기 더보기 클릭
try:
    link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
    while link_more.text == "후기 더보기":
        link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
        # 마지막 후기 더보기까지 눌렀으면 멈추기
        if link_more.text != "후기 접기":
            link_more.click()
            time.sleep(0.3)
# 후기 더보기 없는 경우        
except:
    pass
time.sleep(0.1)


        
        
## 상세page bs4 파싱 분석
soup = bs(driver.page_source, 'html.parser')



In [29]:
# 리뷰 목록 잡기
reveiw_listed = driver.find_element(By.CLASS_NAME, "list_evaluation").find_elements(By.CSS_SELECTOR,'li[data-ismy="false"]')
## 리뷰 내용 더보기 클릭
time.sleep(0.3)
for re_more in range(len(reveiw_listed)):
    try:
        button_more = driver.find_elements(By.CLASS_NAME, 'comment_info')[re_more].find_element(By.CLASS_NAME,'btn_fold')
        button_more.click()
        time.sleep(0.1)
    except:
        pass

In [233]:
driver.find_elements(By.CLASS_NAME, 'comment_info')[21].find_element(By.CLASS_NAME,'btn_fold').click()

In [30]:
len(reveiw_listed)

80

In [217]:
len(driver.find_element(By.CLASS_NAME, "list_evaluation").find_elements(By.CSS_SELECTOR,'li[data-ismy="false"]'))

23

In [157]:
# 웹페이지 닫기
driver.close()

In [31]:
## 매장명
data['merchant'].append(soup.find('h2', {'class':'tit_location'}).get_text())
## 분류
data['category'].append(soup.find('span',{'class':'txt_location'}).get_text().split(':')[1].strip())
## 주소
data['address'].append(soup.find('span', {'class': 'txt_address'}).get_text().replace(" ", "").replace("\n", " "))

## 별점, 리뷰 수 
# 별점 있는 경우
if len(soup.find_all('span',{'class':'color_b'})) > 1:
    data['star'].append(soup.find_all('span', {'class':'color_b'})[0].get_text().strip('점'))
    data['starCount'].append(soup.find('a', {'class': 'link_evaluation'})['data-cnt'])
    data['reviewCount'].append(soup.find_all('span', {'class':'color_b'})[1].get_text().strip('개'))
# 별점 없는 경우
else:
    data['star'].append('0')
    data['starCount'].append('0')
    data['reviewCount'].append(soup.find('span', {'class':'color_b'}).get_text().strip('개'))

## 영업시간
# 더보기 있는 경우
if oper_more:    
    data['oper_time'].append(soup.find_all('ul', {'class':'list_operation'})[1].get_text().strip())
# 더보기 없는 경우
else:
    data['oper_time'].append(soup.find('ul', {'class':'list_operation'}).get_text().strip().replace("\n", ""))

print(f"{i+1}번째 데이터 입력 완료")


15번째 데이터 입력 완료


In [335]:
# 매장명 확인
soup.find('h2', {'class':'tit_location'}).get_text()

'피오니 홍대점'

In [336]:
# 리뷰 목록 잡기
reveiw_listed = soup.find_all('li', {'data-ismy':'false'})
    

In [32]:
data2 = {
    'user_name' : [],
    'user_rank' : [],
    'num_response' : [],
    'user_star' : [], 
    'time' : [],
    'rating' : [],
    'content' : []
}

In [33]:
len(reveiw_listed)

80

In [34]:

for j in range(len(reveiw_listed)):
    ## 유저 이름
    data2['user_name'].append(soup.find_all('span',{'class':'txt_username'})[j].get_text())
    ## 유저 레벨
    data2['user_rank'].append(soup.find_all('span', {'class' : 'txt_badge'})[j].get_text().strip('레벨'))
    ## 유저 후기수
    data2['num_response'].append(
        soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[0].get_text()
    )
    ## 유저 별점 평균
    data2['user_star'].append(
        soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[1].get_text()
    )
    ### 리뷰 작성 시간
    data2['time'].append(
        soup.find_all('div',{'class':'unit_info'})[j].find('span',{'class':'time_write'}).get_text()
    )
    ## 리뷰 내용
    data2['content'].append(
        soup.find_all('p', {'class':'txt_comment'})[j].get_text().strip('더보기')
    )

    ## 유저가 평가한 별점 
    # 너비를 추출하여 별점 계산 (너비는 별점의 백분율을 나타냄)
    style_attribute = soup.find_all('div', {'class':'star_info'})[j].find('span',{'class':'ico_star inner_star'})['style']
    width_percent = float(style_attribute.split(':')[1].strip('%;'))
    rating_out_of_five = width_percent / 20  # 별점은 100%를 5로 나눈 값
    data2['rating'].append(rating_out_of_five)

## 데이터 프레임으로 만들기
df2 = pd.DataFrame(data2)
df2.to_csv(f'./data2/{soup.find('h2', {'class':'tit_location'}).get_text()}.csv',index = False)

In [35]:
print(len(data2['user_name']))
print(len(data2['user_rank']))
print(len(data2['num_response']))
print(len(data2['user_star']))
print(len(data2['time']))
print(len(data2['rating']))
print(len(data2['content']))

80
80
80
80
80
80
80


In [345]:
data2['content']

['예전에는 특별했는데 우유빙수가 흔해지면서부터  굳이 갈 이유가 사라짐 오히려 무스케이크가 맛있는데 너무 비쌈ㅜ',
 '(2월에 방문했습니다)10년전부터 유명한거 알고있었고,늘 사람많고 못가다가이번에 우연히 들렸습니다. 그런데 정말 왜유명한지 모르겠습니다.케이크 생김새만 그럴싸하고맛은 진짜 … 맛없었어요 …살면서 먹어본 케이크중에 젤 별로였어요.. 딸기랑 생크림 진짜 둘다 최악이었음 ..생크림은 제 취향이 아니라쳐도딸기는 그냥 맛없는 딸기였던거같아요저랑 동생이랑 딱 첫입먹자마자맛없다 이랬어요..살면서 케이크가 맛없었던적은 한번도 없었는데처음으로 케이크가 맛없다고 생각했습니다 ..!',
 '딸케 맛있음 과하게 달지않고 크림도 안느끼하고 딱 맛잇엇어용',
 '맛이랑 분위기는 좋은데 비싸',
 '디저트는 그저 그렇고, 커피가 맛있어요!',
 '오랜만에 사먹으니까 가격이 많이 올랐네요.2호 5만원2천원 싼가격은 아닌데이번꺼는 실망스러워요.먹었는데 같이 먹은 일행들 모두시트에서 오래된 맛을 느꼈어요.1호보다 2호는 잘 구입 안하니까 오래 두고 파는거같다는 생각이 드네요.유일하게 케익 사먹는곳인데 다음에 갔을때는 이 점이 개선 되었으면 좋겠네요.',
 '딸기케이크 소문에 비해 별로,, 밍밍한 맛임케익보다 오히려 홍차가 더 나았음',
 '드립커피 맛있음🙂 드립커피라 나올때까지 시간필요😢.옛날부터 딸케 유명 맛집이지만 지난 여름?쯤에 맛이 바뀜... 4명이 모두 딸케를 한 입씩 맛 보고 약품 맛?이 난다고 말함. 사장님께선 바닐라빈을 바꾸었다고 하셨고 친절하게 케이크도 새것으로 다시 내어주셨지만...아무도 손대지 않음..ㅠ',
 '최애딸케집! 크림이 부드럽고 빵이 촉촉합니다 가격도 원랜 비쌌는데 요새 케이크집 생각하면 괜찮은거같아요',
 '가성비가 좋다는 말은 못 하겠는데 딸기빙수가 정말 맛있어요ㅠ 솔직히 딸기 케이크 유명하대서 간건데 딸기케이크는 조금 밍밍해서 별로였습니다 그래도 여름철에 빙수먹으러 가기에는 정말 좋은듯',
 '',
 '분위기도 좋고 딸기케익이 맛있어요!

In [ ]:
data2['rating']